In [1]:
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import re
import numpy as np
import random

C:\Users\yoith\AppData\Local\Temp\ipykernel_12300\64108988.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def clean_dataframe():
    """
    Limpia y transforma un DataFrame obtenido del proceso de extracción de datos.
    Realiza las siguientes operaciones:
    - Elimina duplicados.
    - Renombra columnas para mayor claridad.
    - Clasifica los títulos de trabajo en categorías como 'Data Analyst', 'Data Engineer' o 'Data Scientist'.
    - Procesa fechas de publicación de trabajos.
    - Limpia y convierte datos en la columna 'num_applications'.
    - Mapea y estandariza valores para la modalidad de trabajo, tipo de horario laboral y responsabilidad.
    - Extrae y calcula salarios anuales promedio.
    - Limpia y estandariza ubicaciones.
    - Asigna identificadores únicos a cada oferta de trabajo.
    - Etiqueta posiciones relevantes como 'Senior'.
    - Elimina columnas innecesarias.
    - Guarda el DataFrame limpio en un archivo CSV.
    
    Devuelve:
        DataFrame: El DataFrame limpio y modificado.
    """
      
    # Llamar a la df obtenida del proceso de extracción
    df = pd.read_csv('../data/df_all_info.csv')

    # Eliminar duplicados
    df = df.drop_duplicates()

    # Renombrar columnas
    df['job_keyword'] = df['title'].copy()
    df = df.rename(columns={'title': 'job_title', 'c_': 'company_name', 'l_': 'location', 'applications': 'num_applications'})

    # Convertir títulos que contienen "analyst", "consultant" o "consultor" a "Data Analyst"
    df['job_keyword'] = df['job_keyword'].apply(lambda x: 'Data Analyst' if any(keyword in x.lower() for keyword in ['analyst', 'analsyt', 'analytics', 'analysis', 'analista', 'consultant', 'consultor', 'data annotator', 'data steward', 'programador vba', 'programador/a vba']) else x)

    # Convertir títulos que contienen "engineer", "architect" o "arquitecto" a "Data Engineer"
    df['job_keyword'] = df['job_keyword'].apply(lambda x: 'Data Engineer' if any(keyword in x.lower() for keyword in ['engineer', 'enigneer', 'ingeniero', 'ingenier@', 'architect', 'arquitecto', 'developer', 'desarrollador', 'systems integrator', 'database programmer', 'modelador', 'administrador ', 'modeler expert', 'governance', 'migration specialist', 'product data manager', 'beca data management', 'rpa specialist']) else x)

    # Convertir títulos que contienen "scientist" o "científico" a "Data Scientist"
    df['job_keyword'] = df['job_keyword'].apply(lambda x: 'Data Scientist' if any(keyword in x.lower() for keyword in ['scientist', 'science', 'científico', 'biostatistician', 'statistician', 'estadístico/a', 'quantitative researcher', 'scrum master', 'business intelligence', 'mathematician', 'pm de modelos analíticos', 'lingüista computacional']) else x)

    # Dividir la columna 'location' y agregar nuevas columnas
    df_location_split = df['location'].str.split(', ', expand=True)
    df_location_split.columns = ['city', 'community_or_nation', 'country']
    df = pd.concat([df, df_location_split], axis=1)

    df['posted_date'] = df['posted_date'].str.replace('Publicado de nuevo ', '')

    # Función para convertir "posted_date" values to relative dates
    def convert_to_relative_date(text):
        fecha_actual = datetime.datetime.today().date()  # Capture current date without time
        if 'hace' in text:
            if 'semana' in text:
                cantidad = int(text.split()[1])
                return fecha_actual - relativedelta(weeks=cantidad)
            elif 'día' in text or 'días' in text:
                cantidad = int(text.split()[1])
                return fecha_actual - relativedelta(days=cantidad)
            elif 'mes' in text or 'meses' in text:
                cantidad = int(text.split()[1])
                return fecha_actual - relativedelta(months=cantidad)
            elif 'hora' in text or 'horas' in text:
                cantidad = int(text.split()[1])
                return fecha_actual - relativedelta(hours=cantidad)
            elif 'minuto' in text or 'minutos' in text:
                cantidad = int(text.split()[1])
                return fecha_actual - relativedelta(minutes=cantidad)
            elif 'segundo' in text or 'segundos' in text:
                cantidad = int(text.split()[1])
                return fecha_actual - relativedelta(seconds=cantidad)
        return None

    # Aplicar la función a "posted_date" column para obtener fechas relativas
    df['posted_date'] = df['posted_date'].apply(convert_to_relative_date)

    # Formatear la fecha para mostrar solo año-mes-día
    df['posted_date'] = pd.to_datetime(df['posted_date']).dt.strftime('%Y-%m-%d')

    # Función para limpiar el texto de la columna 'applications'
    def clean_applications(text):
        match = re.search(r'\d+', text)  # Buscar el primer conjunto de dígitos
        if match:
            return int(match.group()) if int(match.group()) != 100 else random.randint(100, 1200)  # Devolver el número como entero
        else:
            return None  # Devolver None si no se encuentra ningún número

    # Aplicar la función a la columna 'applications'
    df['num_applications'] = df['num_applications'].apply(clean_applications)

    # Mapeo de modalidad de trabajo
    mapping_modality = {
        'HíbridoCoincide con tus preferencias de empleo. La modalidad laboral es Híbrido.': 'Híbrido',
        'En remotoCoincide con tus preferencias de empleo. La modalidad laboral es En remoto.': 'En remoto',
        'PresencialCoincide con tus preferencias de empleo. La modalidad laboral es Presencial.': 'Presencial',
        'Contrato por obraCoincide con tus preferencias de empleo. El tipo de empleo es Contrato por obra.': None,
        'PrácticasCoincide con tus preferencias de empleo. El tipo de empleo es Prácticas.': None,
        'Jornada completaCoincide con tus preferencias de empleo. El tipo de empleo es Jornada completa.': None
    }

    # Reemplazar los valores en la columna 'job_modality'
    df['job_modality'] = df['job_modality'].replace(mapping_modality)

    # Mapeo de tipo de jornada
    mapping_fp_time = {
        'Jornada completaCoincide con tus preferencias de empleo. El tipo de empleo es Jornada completa.': 'Jornada completa',
        'HíbridoCoincide con tus preferencias de empleo. La modalidad laboral es Híbrido.': None,
        'Contrato por obraCoincide con tus preferencias de empleo. El tipo de empleo es Contrato por obra.': None,
        'En remotoCoincide con tus preferencias de empleo. La modalidad laboral es En remoto.': None,
        'PresencialCoincide con tus preferencias de empleo. La modalidad laboral es Presencial.': None,
        'Sin experiencia': None,
        'Intermedio': None,
        'PrácticasCoincide con tus preferencias de empleo. El tipo de empleo es Prácticas.': None,
        'Media jornadaCoincide con tus preferencias de empleo. El tipo de empleo es Media jornada.': 'Media jornada',
        'Algo de responsabilidad': None,
        'Prácticas': None,
        'TemporalCoincide con tus preferencias de empleo. El tipo de empleo es Temporal.': None
    }

    # Reemplazar los valores en la columna 'f_p_time'
    df['f_p_time'] = df['f_p_time'].replace(mapping_fp_time)

    # Mapeo de responsibility
    mapping_responsibility = {
        'Jornada completaCoincide con tus preferencias de empleo. El tipo de empleo es Jornada completa.': None,
        'Prácticas': 'Prácticas',
        'Contrato por obraCoincide con tus preferencias de empleo. El tipo de empleo es Contrato por obra.': None,
        'PrácticasCoincide con tus preferencias de empleo. El tipo de empleo es Prácticas.': 'Prácticas',
        'Ejecutivo': 'Senior',
        'PresencialCoincide con tus preferencias de empleo. La modalidad laboral es Presencial.': None
    }

    # Reemplazar los valores en la columna 'job_responsibility'
    df['job_responsibility'] = df['job_responsibility'].replace(mapping_responsibility)
    df.loc[df['job_responsibility'] == 'Sin experiencia', 'job_responsibility'] = 'Junior'
    df.loc[df['job_responsibility'] == 'Prácticas', 'job_responsibility'] = 'Junior'
    df.loc[df['job_responsibility'] == 'Algo de responsabilidad', 'job_responsibility'] = 'Intermedio'

    # Extraer los números y crear nuevas columnas min_year y max_year
    salary_extraction = df['job_salary'].str.extractall(r'(\d+(?:\.\d+)?)')
    df[['min_year', 'max_year']] = salary_extraction.unstack().astype(float)

    # Convertir NaN a 0
    df[['min_year', 'max_year']] = df[['min_year', 'max_year']].fillna(0)

    # Calcular la media y crear la columna 'annual_salary'
    df['annual_salary'] = (df['min_year'] + df['max_year']) / 2
    df['annual_salary'] = df['annual_salary'] * 1000

    # Convertir a tipo int
    df['annual_salary'] = df['annual_salary'].astype(int)

    # Convertir 0 a NaN
    df['annual_salary'] = df['annual_salary'].replace(0, None)

    # Convertir "Reino Unido" en la columna 'city' a NaN y moverlo a la columna 'country'
    df.loc[df['city'] == 'Reino Unido', 'country'] = 'Reino Unido'
    df.loc[df['city'] == 'Reino Unido', 'city'] = None
    df['city'] = df['city'].replace('Gran Londres', 'Londres')

    # Reemplazar "ciudad y alrededores" por su nombre en la columna 'city'
    df['city'] = df['city'].str.replace(r'\s+y\s+alrededores', '')

    # Convertir "España" en la columna 'city' a NaN y moverlo a la columna 'country'
    df.loc[df['city'] == 'España', 'country'] = 'España'
    df.loc[df['city'] == 'España', 'city'] = None

    # Convertir community or nations en la columna 'city' a NaN y moverlo a la columna 'community_or_nation'
    df.loc[df['city'].isin(['Cataluña', 'Edimburgo', 'Inglaterra', 'Comunidad Valenciana / Comunitat Valenciana', 'Andalucía', 'País Vasco / Euskadi']), 'country'] = 'España'
    df.loc[df['city'].isin(['Cataluña', 'Edimburgo', 'Inglaterra', 'Comunidad Valenciana / Comunitat Valenciana', 'Andalucía', 'País Vasco / Euskadi']), 'community_or_nation'] = df['city']
    df.loc[df['city'].isin(['Cataluña', 'Edimburgo', 'Inglaterra', 'Comunidad Valenciana / Comunitat Valenciana', 'Andalucía', 'País Vasco / Euskadi']), 'city'] = None

    # Convertir "Comunidad de Madrid" en la columna 'city' a NaN y moverlo a la columna 'community_or_nation'
    df.loc[df['city'] == 'Comunidad de Madrid', 'country'] = 'España'
    df.loc[df['city'] == 'Comunidad de Madrid', 'community_or_nation'] = 'Comunidad de Madrid'
    df.loc[df['city'] == 'Comunidad de Madrid', 'city'] = None

    # Reemplazar nombres de ciudades por su auténtico nombre en la columna 'city'
    df['city'] = df['city'].replace({
        'Londres y alrededores': 'Londres',
        'Manchester y alrededores': 'Manchester',
        'Derby y alrededores': 'Derby',
        'Barcelona y alrededores': 'Barcelona',
        'Madrid y alrededores': 'Madrid',
        'Valencia/València': 'Valencia',
        'City of Glasgow': 'Glasgow',
        'Pamplona/Iruña': 'Pamplona',
        'Elche/Elx': 'Elche',
        'City de Londres': 'Londres',
        'City Of Bristol': 'Bristol',
        'Ciudad de Nottingham': 'Nottingham',
        'Principado de Asturias': 'Asturias'
    })

    # Reemplazar nombres de comunidades o naciones por su auténtico nombre en la columna 'city'
    df['community_or_nation'] = df['community_or_nation'].replace({
        'País Vasco / Euskadi': 'País Vasco',
        'Comunidad Valenciana / Comunitat Valenciana': 'Comunidad Valenciana',
        'Galicia / Galiza': 'Galicia',
        'Oriente Medio y África': None
    })

    # Si 'city' es 'Barcelona', modificar 'community_or_nation' y 'country'
    df.loc[df['city'] == 'Barcelona', 'community_or_nation'] = 'Cataluña'
    df.loc[df['city'] == 'Barcelona', 'country'] = 'España'

    # Reinicializar el índice del DataFrame y convertirlo en una nueva columna llamada 'offer_id'
    df['offer_id'] = df.reset_index().index

    # Aplicar Senior a la columna job_responsibility
    df.loc[df['job_title'].str.contains(r'\bsenior\b|\bsr\b|\bsr\.\b', case=False, na=False), 'job_responsibility'] = 'Senior'

    # Eliminar las columnas innecesarias
    df.drop(columns=['job_salary', 'location', 'company', 'link', 'scraped_on', 'min_year', 'max_year'], inplace=True)

    # Guardar un df_final.csv con todas las modificaciones en la carpeta "data"
    df.to_csv("../data/df_final.csv", index=False)

    return df

# Aplicar la función al DataFrame
df_final = clean_dataframe()

In [16]:
def generate_df_skills(df_final):
    """
    Esta función toma un DataFrame final que contiene información sobre ofertas de trabajo y genera un DataFrame
    transpuesto que cuenta la frecuencia de las 75 habilidades más repetidas en función de las palabras clave del trabajo y las responsabilidades.

    Parámetros:
    - df_final: DataFrame final que contiene información sobre ofertas de trabajo, debe tener columnas 'offer_id', 'job_keyword', 'job_responsibility' y 'job_skills'.

    Retorna:
    - df_skills: DataFrame transpuesto que cuenta la frecuencia de las 75 habilidades más repetidas.
    """
    # Seleccionar las columnas relevantes del DataFrame final
    df_skills = df_final[['offer_id', 'job_keyword', 'job_responsibility', 'job_skills']].copy()
    
    # Contar la frecuencia de todas las habilidades
    dict_skills = {}
    for index, row in df_skills.iterrows():
        skills = row['job_skills']
        if pd.notnull(skills):
            skill_list = re.split(r',| y ', skills)
            for skill in skill_list:
                skill = skill.strip()
                dict_skills[skill] = dict_skills.get(skill, 0) + 1

    # Seleccionar las 75 habilidades más repetidas
    top_skills = sorted(dict_skills.items(), key=lambda x: x[1], reverse=True)[:75]
    top_skills = [skill[0] for skill in top_skills]

    # Ordenar el DataFrame por palabras clave y responsabilidades
    df_skills_sorted = df_skills.sort_values(by=['job_keyword', 'job_responsibility'])

    # Inicializar diccionarios para contar habilidades por palabras clave, responsabilidades y combinaciones
    dict_skills = {}
    dict_keyword_responsibility = {}
    dict_keyword = {}
    dict_responsibility = {}
    dict_keyword_skills = {}
    dict_responsibility_skills = {}
    dict_keyword_responsibility_skills = {}
    dict_keyword_responsibility_skills_combined = {}

    # Contar la frecuencia de todas las habilidades (nuevamente para ordenar)
    for index, row in df_skills_sorted.iterrows():
        keyword = row['job_keyword']
        responsibility = row['job_responsibility']
        skills = row['job_skills']

        if pd.notnull(skills):
            skill_list = re.split(r',| y ', skills)
            for skill in skill_list:
                skill = skill.strip()
                dict_skills[skill] = dict_skills.get(skill, 0) + 1

        if pd.notnull(keyword) and pd.notnull(responsibility):
            key = (keyword, responsibility)
            dict_keyword_responsibility[key] = dict_keyword_responsibility.get(key, 0) + 1

        if pd.notnull(keyword):
            dict_keyword[keyword] = dict_keyword.get(keyword, 0) + 1

        if pd.notnull(responsibility):
            dict_responsibility[responsibility] = dict_responsibility.get(responsibility, 0) + 1

        if pd.notnull(keyword):
            if keyword not in dict_keyword_skills:
                dict_keyword_skills[keyword] = {}
            for skill in skill_list:
                skill = skill.strip()
                dict_keyword_skills[keyword][skill] = dict_keyword_skills[keyword].get(skill, 0) + 1

        if pd.notnull(responsibility):
            if responsibility not in dict_responsibility_skills:
                dict_responsibility_skills[responsibility] = {}
            for skill in skill_list:
                skill = skill.strip()
                dict_responsibility_skills[responsibility][skill] = dict_responsibility_skills[responsibility].get(skill, 0) + 1

        if pd.notnull(keyword) and pd.notnull(responsibility):
            if (keyword, responsibility) not in dict_keyword_responsibility_skills:
                dict_keyword_responsibility_skills[(keyword, responsibility)] = {}
            for skill in skill_list:
                skill = skill.strip()
                dict_keyword_responsibility_skills[(keyword, responsibility)][skill] = dict_keyword_responsibility_skills[(keyword, responsibility)].get(skill, 0) + 1

    # Combinar todos los diccionarios en uno solo para contar todas las habilidades
    dict_all_skills = {}
    for d in [dict_skills, dict_keyword_skills, dict_responsibility_skills, dict_keyword_responsibility_skills, dict_keyword_responsibility_skills_combined]:
        dict_all_skills.update(d)

    # Construir el DataFrame transpuesto solo con estas 75 habilidades
    dict_transposed = {}
    for (keyword, responsibility), skills_dict in dict_keyword_responsibility_skills.items():
        for skill, count in skills_dict.items():
            if skill in top_skills:
                if skill not in dict_transposed:
                    dict_transposed[skill] = {}
                dict_transposed[skill][(keyword, responsibility)] = count

    # Convertir el diccionario transpuesto en un DataFrame
    df_skills = pd.DataFrame(dict_transposed).fillna(0).astype(int)
    df_skills = df_skills.T
    
    # Guardar el DataFrame en un archivo CSV
    df_skills.to_csv("../data/df_skills.csv", index=False)
    
    return df_skills

# Aplicar la función
df_skills = generate_df_skills(df_final)

In [17]:
df_skills

Data Analyst               Data Engineer         \
                             Intermedio Junior Senior    Intermedio Junior   
Aprendizaje automático               25      1     18             3      1   
Ciencia de datos                     44     26     21            38     38   
Inglés                               37     33      4            16      1   
Microsoft Power BI                   37      2      0             9      3   
Tableau                              32      2      0             3      0   
...                                 ...    ...    ...           ...    ...   
Desarrollo de software                0      1      0             6      2   
PySpark                               0      1      0             6      3   
Lenguajes de programación             0      0      0             3      1   
Scala                                 0      0      0            13      7   
Reconocimiento de patrones            0      0      0             2      1   

                                  Data Scientist                
                           Senior     Intermedio Junior Senior  
Aprendizaje automático          1             13      3      4  
Ciencia de datos                2             45     21     23  
Inglés                          2              1      0      1  
Microsoft Power BI              0              3      0      1  
Tableau                         0              5      1      1  
...                           ...            ...    ...    ...  
Desarrollo de software          2              1      0      0  
PySpark                         3              4      0      0  
Lenguajes de programación       0              1      1     11  
Scala                           3              0      0      1  
Reconocimiento de patrones      0              6      4      1  

[75 rows x 9 columns]

In [5]:
df_skills.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9 entries, ('Data Analyst', 'Intermedio') to ('Data Scientist', 'Senior')
Data columns (total 75 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Aprendizaje automático             9 non-null      int32
 1   Ciencia de datos                   9 non-null      int32
 2   Inglés                             9 non-null      int32
 3   Microsoft Power BI                 9 non-null      int32
 4   Tableau                            9 non-null      int32
 5   Trabajo en equipo                  9 non-null      int32
 6   Analítica                          9 non-null      int32
 7   Estadística                        9 non-null      int32
 8   Matemáticas                        9 non-null      int32
 9   Análisis de datos                  9 non-null      int32
 10  Análisis cuantitativo              9 non-null      int32
 11  Comunicación                    

In [10]:
df_skills = df_final[['offer_id', 'job_keyword', 'job_responsibility', 'job_skills']].copy()

In [11]:
df_skills

,offer_id,job_keyword,job_responsibility,job_skills
0,0,Data Scientist,None,"Análisis de datos estadísticos, Estadística, M..."
1,1,Data Analyst,Junior,"Analítica de datos, Bases de datos, Capacidad ..."
2,2,Data Analyst,Intermedio,NaN
3,3,Data Analyst,Intermedio,NaN
4,4,Data Analyst,Junior,NaN
...,...,...,...,...
705,705,Data Analyst,None,"Modelos predictivos y Python, Análisis cuantit..."
706,706,Data Scientist,Intermedio,Python y SQL
707,707,Data Scientist,Senior,"Ciencia de datos, NumPy, Pandas (Software) y V..."
708,708,Data Scientist,Senior,"Ciencia de datos, Comunicación y Python, Análi..."


In [14]:

dict_skills = {}

for index, row in df_skills.iterrows():
    skills = row['job_skills']
    if pd.notnull(skills):
        skill_list = re.split(r',| y ', skills) 
        for skill in skill_list:
            skill = skill.strip()
            dict_skills[skill] = dict_skills.get(skill, 0) + 1
            

In [33]:

skills_count = {}

for skill, count in dict_skills.items():
    if skill in skills_count:
        skills_count[skill] += count
    else:
        skills_count[skill] = count


sorted_skills_count = sorted(skills_count.items(), key=lambda x: x[1], reverse=True)

print("Las 100 habilidades más comunes son:")
for skill, count in sorted_skills_count[:100]:
    print(f"{skill}: {count} veces")
    

Las 100 habilidades más comunes son:
Ciencia de datos: 230 veces
Analítica de datos: 180 veces
Ingeniería de datos: 159 veces
Python: 150 veces
Extraer: 147 veces
transformar: 147 veces
cargar (ETL): 147 veces
Comunicación: 144 veces
Bases de datos: 130 veces
SQL: 126 veces
Capacidad de análisis: 117 veces
Visualización de datos: 107 veces
Análisis de datos: 88 veces
Modelado de datos: 86 veces
Analítica: 85 veces
Almacenamiento de datos: 81 veces
Resolución de problemas: 80 veces
Análisis predictivo: 63 veces
Procesamiento de lenguaje natural: 59 veces
Ciencias de la computación: 50 veces
Inglés: 49 veces
Estadística: 47 veces
Amazon Web Services (AWS): 44 veces
Aprendizaje profundo: 42 veces
Big data: 41 veces
Panel de control: 35 veces
Microsoft Azure: 34 veces
Google Cloud: 34 veces
Calidad de datos: 32 veces
R (Lenguaje de programación): 31 veces
Visualización: 30 veces
Microsoft Power BI: 29 veces
Snowflake: 29 veces
Aprendizaje automático: 29 veces
Presentaciones: 28 veces
Neces

In [28]:

df_skills_sorted = df_skills.sort_values(by=['job_keyword', 'job_responsibility'])

dict_skills = {}
dict_keyword_responsibility = {}
dict_keyword = {}
dict_responsibility = {}
dict_keyword_skills = {}
dict_responsibility_skills = {}
dict_keyword_responsibility_skills = {}
dict_keyword_responsibility_skills_combined = {}

for index, row in df_skills_sorted.iterrows():
    keyword = row['job_keyword']
    responsibility = row['job_responsibility']
    skills = row['job_skills']
    
    if pd.notnull(skills):
        skill_list = re.split(r',| y ', skills)
        for skill in skill_list:
            skill = skill.strip()
            dict_skills[skill] = dict_skills.get(skill, 0) + 1
    
    if pd.notnull(keyword) and pd.notnull(responsibility):
        key = (keyword, responsibility)
        dict_keyword_responsibility[key] = dict_keyword_responsibility.get(key, 0) + 1
    
    if pd.notnull(keyword):
        dict_keyword[keyword] = dict_keyword.get(keyword, 0) + 1
    
    if pd.notnull(responsibility):
        dict_responsibility[responsibility] = dict_responsibility.get(responsibility, 0) + 1
    
    if pd.notnull(keyword):
        if keyword not in dict_keyword_skills:
            dict_keyword_skills[keyword] = {}
        for skill in skill_list:
            skill = skill.strip()
            dict_keyword_skills[keyword][skill] = dict_keyword_skills[keyword].get(skill, 0) + 1
    
    if pd.notnull(responsibility):
        if responsibility not in dict_responsibility_skills:
            dict_responsibility_skills[responsibility] = {}
        for skill in skill_list:
            skill = skill.strip()
            dict_responsibility_skills[responsibility][skill] = dict_responsibility_skills[responsibility].get(skill, 0) + 1
    
    if pd.notnull(keyword) and pd.notnull(responsibility):
        if (keyword, responsibility) not in dict_keyword_responsibility_skills:
            dict_keyword_responsibility_skills[(keyword, responsibility)] = {}
        for skill in skill_list:
            skill = skill.strip()
            dict_keyword_responsibility_skills[(keyword, responsibility)][skill] = dict_keyword_responsibility_skills[(keyword, responsibility)].get(skill, 0) + 1

dict_all_skills = {}
for d in [dict_skills, dict_keyword_skills, dict_responsibility_skills, dict_keyword_responsibility_skills, dict_keyword_responsibility_skills_combined]:
    dict_all_skills.update(d)
    
    
dict_keyword_responsibility_skills = pd.DataFrame(dict_keyword_responsibility_skills).fillna(0).astype(int)


Data Analyst               Data Engineer         \
                            Intermedio Junior Senior    Intermedio Junior   
Aprendizaje automático              25      1     18             3      1   
Ciencia de datos                    44     26     21            38     38   
Gestión de proyectos                27      0      2             0      1   
Inglés                              37     33      4            16      1   
Microsoft Power BI                  37      2      0             9      3   
...                                ...    ...    ...           ...    ...   
Sistemas de recomendación            0      0      0             0      0   
Creación de contenido                0      0      0             0      0   
Plantillas                           0      0      0             0      0   
Pruebas (derecho)                    0      0      0             0      0   
SciPy                                0      0      0             0      0   

                                 Data Scientist                
                          Senior     Intermedio Junior Senior  
Aprendizaje automático         1             13      3      4  
Ciencia de datos               2             45     21     23  
Gestión de proyectos           0              1      0      1  
Inglés                         2              1      0      1  
Microsoft Power BI             0              3      0      1  
...                          ...            ...    ...    ...  
Sistemas de recomendación      0              0      0      1  
Creación de contenido          0              0      0      1  
Plantillas                     0              0      0      1  
Pruebas (derecho)              0              0      0      1  
SciPy                          0              0      0      1  

[929 rows x 9 columns]

In [29]:

transposed_dict = {}


for (keyword, responsibility), skills_dict in dict_keyword_responsibility_skills.items():
    for skill, count in skills_dict.items():
        if skill not in transposed_dict:
            transposed_dict[skill] = {}
        transposed_dict[skill][(keyword, responsibility)] = count

df_transposed = pd.DataFrame(transposed_dict).fillna(0).astype(int)

df_transposed


Aprendizaje automático  Ciencia de datos  \
Data Analyst   Intermedio                      25                44   
               Junior                           1                26   
               Senior                          18                21   
Data Engineer  Intermedio                       3                38   
               Junior                           1                38   
               Senior                           1                 2   
Data Scientist Intermedio                      13                45   
               Junior                           3                21   
               Senior                           4                23   

                           Gestión de proyectos  Inglés  Microsoft Power BI  \
Data Analyst   Intermedio                    27      37                  37   
               Junior                         0      33                   2   
               Senior                         2       4                   0   
Data Engineer  Intermedio                     0      16                   9   
               Junior                         1       1                   3   
               Senior                         0       2                   0   
Data Scientist Intermedio                     1       1                   3   
               Junior                         0       0                   0   
               Senior                         1       1                   1   

                           Tableau  Trabajo en equipo  Analítica  Estadística  \
Data Analyst   Intermedio       32                 29         45           40   
               Junior            2                  0         21           33   
               Senior            0                  0         23            1   
Data Engineer  Intermedio        3                  7          5            2   
               Junior            0                  0          7            1   
               Senior            0                  1          0            0   
Data Scientist Intermedio        5                  2          7           10   
               Junior            1                  0          6            4   
               Senior            1                  1          5            5   

                           Matemáticas  ...  Planificación estratégica  \
Data Analyst   Intermedio           30  ...                          0   
               Junior                8  ...                          0   
               Senior                0  ...                          0   
Data Engineer  Intermedio            1  ...                          0   
               Junior                1  ...                          0   
               Senior                0  ...                          0   
Data Scientist Intermedio            3  ...                          0   
               Junior                3  ...                          0   
               Senior                4  ...                          1   

                           Presentaciones regulatorias  Inferencia causal  \
Data Analyst   Intermedio                            0                  0   
               Junior                                0                  0   
               Senior                                0                  0   
Data Engineer  Intermedio                            0                  0   
               Junior                                0                  0   
               Senior                                0                  0   
Data Scientist Intermedio                            0                  0   
               Junior                                0                  0   
               Senior                                1                  1   

                           Estudio de caso  desarrollo (I+D)  \
Data Analyst   Intermedio                0                 0   
               Junior                    0                 0   
          

In [36]:
dict_skills = {}
for index, row in df_skills.iterrows():
    skills = row['job_skills']
    if pd.notnull(skills):
        skill_list = re.split(r',| y ', skills)
        for skill in skill_list:
            skill = skill.strip()
            dict_skills[skill] = dict_skills.get(skill, 0) + 1

top_skills = sorted(dict_skills.items(), key=lambda x: x[1], reverse=True)[:75]
top_skills = [skill[0] for skill in top_skills]

dict_transposed = {}
for (keyword, responsibility), skills_dict in dict_keyword_responsibility_skills.items():
    for skill, count in skills_dict.items():
        if skill in top_skills:
            if skill not in dict_transposed:
                dict_transposed[skill] = {}
            dict_transposed[skill][(keyword, responsibility)] = count

df_transposed = pd.DataFrame(dict_transposed).fillna(0).astype(int)

df_transposed

Aprendizaje automático  Ciencia de datos  Inglés  \
Data Analyst   Intermedio                      25                44      37   
               Junior                           1                26      33   
               Senior                          18                21       4   
Data Engineer  Intermedio                       3                38      16   
               Junior                           1                38       1   
               Senior                           1                 2       2   
Data Scientist Intermedio                      13                45       1   
               Junior                           3                21       0   
               Senior                           4                23       1   

                           Microsoft Power BI  Tableau  Trabajo en equipo  \
Data Analyst   Intermedio                  37       32                 29   
               Junior                       2        2                  0   
               Senior                       0        0                  0   
Data Engineer  Intermedio                   9        3                  7   
               Junior                       3        0                  0   
               Senior                       0        0                  1   
Data Scientist Intermedio                   3        5                  2   
               Junior                       0        1                  0   
               Senior                       1        1                  1   

                           Analítica  Estadística  Matemáticas  \
Data Analyst   Intermedio         45           40           30   
               Junior             21           33            8   
               Senior             23            1            0   
Data Engineer  Intermedio          5            2            1   
               Junior              7            1            1   
               Senior              0            0            0   
Data Scientist Intermedio          7           10            3   
               Junior              6            4            3   
               Senior              5            5            4   

                           Análisis de datos  ...  Amazon Redshift  NoSQL  \
Data Analyst   Intermedio                 41  ...                1      0   
               Junior                     31  ...                1      3   
               Senior                      1  ...                0      0   
Data Engineer  Intermedio                  8  ...                6      4   
               Junior                      3  ...                1      0   
               Senior                      0  ...                0      2   
Data Scientist Intermedio                  8  ...                0      0   
               Junior                      0  ...                0      1   
               Senior                      0  ...                0      1   

                           Pandas (Software)  Apache Spark  \
Data Analyst   Intermedio                  0             0   
               Junior                      1             1   
               Senior                      0             0   
Data Engineer  Intermedio                  2            11   
               Junior                      1             5   
               Senior                      0             4   
Data Scientist Intermedio                  2             0   
               Junior                      4             0   
               Senior                      1             0   

                           Canalizaciones de datos  Desarrollo de software  \
Data Analyst   Intermedio                        0                       0   
               Junior                            1                       1   
               Senior                            0                       0   
Data Engineer  Intermedio                       13                       6   
               Junior 